# MLflow setup

**MFflow Tracking** permite organizar la información de los entrenamientos de modelos, con total trazabilidad de modelos, parámetros, métricas, ficheros, etc.

MFflow Tracking guarda información en una carpeta local. En usa empresa se usaría el módulo **MLflow Registry** como un repositório central de modelos, para que otros miembros del equipo tuvieran acceso a los modelos entrenados y guardados en ese Registry.

![Uso local de MLflow](img/mlflow_localhost.png)

MLflow Tracking crea dos instancias:
- LocalArtifactRepository (para guardar artifacts)
- FileStore (para guardar entities)

## Conceptos
- **run** corresponde a la ejecución de código de data science. Para  cada run ser registra:
 - commit del código (git)
 - hora de inicio y fin del run
 - source (fichero o nombre del proyecto MLflow que dispara el run)
 - parámetros (tanto los del modelo como los definidos por nosotros)
 - métricas
 - artefactos (cualquier fichero que querramos, por ej. ficheros de salida, modelos, etc.)

Se pueden crear **experiments** (opcional), que agrupan runs que ejecutamos para una tarea concreta.

## Instalación

In [1]:
# !pip install mlflow

## Interfaz grafico

### Instalación
Para activar el interfaz grafico se usa el comando `mlflow ui` (en el terminal del contenedor).

La versión anterior del script de arranque del contenedor (en `/env/start.sh`) no exponía el puerto 5000.

#### Si el contenedor Docker nunca ha sido arrancado

Usar contenedores requiere Docker Desktop instalado en la máquina (host).

1. En la línea de comando, arrancar el contenedor con el comando `./env/start.sh`
2. Abrir un terminal en el contenedor con el comando `docker exec -it ds-TF_MDS /bin/bash`
3. En el terminal del contenedor, arrancar el frontal de MLflow con el comando `mlflow ui`
4. En el host abrir un navegador e ir a `http://localhost:5050` (o `http://127.0.0.1:5050`)

#### Si el contenedor Docker ya había arrancado anteriormente

1. Guardar todos los nobebooks y scripts Python abiertos en el contenedor
2. Parar y eliminar contenedor `docker rm -f ds-TF_MDS` (solo se pierden las librerías que habíamos instalado manualmente en el contenedor)
3. Seguir los pasos arriba

### Visualizar métricas
- [documentación](https://mlflow.org/docs/latest/tracking.html#visualizing-metrics)
- Acceso al [interfaz grafico](http://127.0.0.1:5050) de MLflow (botón derecho > abrir en nueva pestaña)

![Visualizar métricas en el ui](img/metrics-step.png)

## Experimentos

### Crear un experimento

In [ ]:
import mlflow
from pathlib import Path

# Create an experiment name, which must be unique and case sensitive
experiment_id = mlflow.create_experiment(
    "Entrenamiento de CNN",
    artifact_location=Path.cwd().joinpath("mlruns").as_uri(),
    tags={"version": "v1", "priority": "P1"},
)
experiment = mlflow.get_experiment(experiment_id)

### Taguear experimento

In [ ]:
# un solo tag
mlflow.set_experiment_tag("release.version", # tag name
                          "2.2.0") # tag value

# más de uno
tags = {"engineering": "ML Platform",
        "release.candidate": "RC1",
        "release.version": "2.2.0"}
mlflow.set_experiment_tags(tags)

### Eliminar experimento

In [ ]:
mlflow.delete_experiment(experiment_id)

# Examine the deleted experiment details.
experiment = mlflow.get_experiment(experiment_id)
print("Name: {}".format(experiment.name))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

### Listar experimentos

In [ ]:
mlflow.list_experiments()

## Runs

### Crear run

In [ ]:
import mlflow

with mlflow.start_run(
    run_name="PARENT_RUN", # solo si no indicamos run_id
    experiment_id=experiment_id,
    tags={"version": "v1", "priority": "P1"},
    description="parent",
) as run:
    # modedelar y loguear

### Reanudar run

In [ ]:
with mlflow.start_run(
    run_id=run_id # obtenido con run.info.run_id
) as run:
    # modedelar y loguear

### Taguear run
**Atención:** si no hay run en curso, crea uno.

In [ ]:
# uno
mlflow.set_tag("release.version", "2.2.0")

# más de uno
tags = {"engineering": "ML Platform",
        "release.candidate": "RC1",
        "release.version": "2.2.0"}
mlflow.set_tags(tags)

### Ver info del run

In [ ]:
run_id = run.info.run_id
run = mlflow.active_run()
print("run_id: {}; status: {}".format(run.info.run_id, run.info.status))

### Terminar run

In [ ]:
mlflow.end_run()
run = mlflow.get_run(run.info.run_id)
print("run_id: {}; status: {}".format(run.info.run_id, run.info.status))
print("--")

# Check for any active runs
print("Active run: {}".format(mlflow.active_run()))

### Ver información de los runs

In [ ]:
print_run_infos(mlflow.list_run_infos("0", run_view_type=ViewType.ALL))

### Apagar run

In [ ]:
mlflow.delete_run(run_id)

print("run_id: {}; lifecycle_stage: {}".format(run_id,
    mlflow.get_run(run_id).info.lifecycle_stage))

## Logging

### Auto-logging

Loguea información automaticamente, en función de la librería de modelación que estemos utilizando.

Atualmente soporta:
- Scikit-learn
- TensorFlow
- Keras
- Gluon
- XGBoost
- LightGBM
- Statsmodels
- Spark
- Fastai
- Pytorch

In [ ]:
# a continuación se indican valores por defecto para el autolog genérico
mlflow.autolog(log_input_examples = False, # guardar ejemplos (si el modelo así lo contempla - p.ej. en transformers)
               log_model_signatures = True,# signatures son descripciones de las entradas y salidas del modelo model
                                           # durante entrenamiento. Solo se generan si log_models = True
               log_models = True, # guarda modelo como artefacto
               disable = False, # activa auto logging (True desactiva)
               exclusive = False, # no loguea "user-created fluent runs" (?)
               disable_for_unsupported_versions = False, # solo auto-loguea librerías soportadas
               silent = False) # incluir event logs y warnings en el log (True excluye)

#### Autolog con Keras
- [Documentación](https://mlflow.org/docs/latest/tracking.html#tensorflow-and-keras)

Se guardan:
 - **Métricas:**
   - Training loss, validation loss y métricas definidas por el usuario.
   - Métricas de EarlyStopping. Por ejemplo: stopped_epoch, restored_epoch, restore_best_weight, etc.
 - **Parámetros:**
   - paámetros de fit(), nombre del optimizador, learning rate y epsilon.
   - Parámetros asoociados al EarlyStopping. Por ejemplo: min_delta, patience, baseline, restore_best_weights, etc.
 - **Modelo:**
   - summary del modelo MLflow (modelo Keras) al iniciar y al terminar el entrenamiento

In [ ]:
# Meter antes del cógido del modelo, para loguear automaticamente las métricas y parámetros
mlflow.tensorflow.autolog()

### Loguear artefactos (ficheros)

In [ ]:
# ejemplo para loguear ficheros en /data
mlflow.log_artifacts("data", # carpeta local donde están los ficheros a loguear
                     artifact_path="states") # (opcional) carpeta dentro de artifact_uri en que se guardarán los ficheros

### Loguear dicionario

In [ ]:
dictionary = {"k": "v"}
# Log a dictionary as a JSON file under the run's root artifact directory
mlflow.log_dict(dictionary, "data.json")

### Loguear figura

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot([0, 1], [2, 3])

mlflow.log_figure(fig, "figure.png")

### Loguear imagen (ndarray o PIL)
**Atención:** limita interos en [0, 255] y float en [0, 1]. Valores menores/mayores son ajustados y los demás no cambian, por lo que puede introdución una reducción de la escala dinámica de las fotos.

In [ ]:
mlflow.log_image(image, "image.png")

### Loguear métricas

In [ ]:
# una
mlflow.log_metric("mse", # nombre de la métrica
                  2500.00) # valor

# más de una
metrics = {"mse": 2500.00,
           "rmse": 50.00}
mlflow.log_metrics(metrics)

### Loguear parámetros
**Atención:** si no hay run en curso, crea uno.

In [ ]:
# uno
mlflow.log_param("learning_rate", # key
                 0.01) # value

# más de uno
params = {"learning_rate": 0.01, "n_estimators": 10}
mlflow.log_params(params)

### Loguear texto en un fichero (artefacto)

In [ ]:
mlflow.log_text("text1", # texto
                "file1.txt") # artefacto, en la carpeta de artefactos del run

## Interfaz grafico
### Runs de un experimento
![Runs de experimento](img/screenshot.JPG)]

## Parámetros, métricas y artefactos de un run

![Items logueados](img/screenshot2.JPG)